In [1]:
import pandas as pd
import numpy as np
import re

# Which bonds are green?

In [2]:
# reading data of which bonds are held by ECB at 2nd of april 2021
holdings20210402 = pd.read_csv("data/CSPP_PEPP_corporate_bond_holdings_20210402.csv", header=0, encoding='latin-1')
holdings20210402

,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE
0,BE,BE0002239086,Elia Transmission Belgium,27/05/2024,1.375
1,BE,BE0002256254,Enodia SCRL,22/07/2026,1
2,BE,BE0002276450,Elia Transmission Belgium,07/04/2027,1.375
3,BE,BE0002280494,Groupe Bruxelles Lambert SA,23/05/2024,1.375
4,BE,BE0002285543,Fluvius System Operator CVBA,23/06/2025,2
...,...,...,...,...,...
1637,IT,XS2292487076,ACEA S.p.A.,28/07/2030,0.25
1638,IT,XS2292547317,2i Rete Gas S.p.A.,29/01/2031,0.579
1639,IT,XS2299001888,Italgas S.P.A.,16/02/2028,0
1640,IT,XS2299002423,Italgas S.P.A.,16/02/2033,0.5


In [3]:
# reading data of all green bonds in the world (ICMA - 06 april 2021)
greenBonds = pd.read_csv("data/ICMA-Sustainable-Bonds-Database-060421.csv", delimiter=";", encoding="latin-1")
greenBonds

,Green Bond issuer,Country,Market Information Template,External Review Form,External Review Report,External links (NB: Please use the links for most up to date information),Unnamed: 6,Unnamed: 7
0,ABN AMRO (2015),Netherlands,NaN,NaN,oekom research,https://www.abnamro.com/en/investor-relations/...,NaN,NaN
1,ABN AMRO (2016),Netherlands,NaN,NaN,oekom research,https://www.abnamro.com/en/investor-relations/...,NaN,NaN
2,AC Energy Finance Supra-national,Philippines,NaN,Appendix to Sustainalytics report,SUSTAINALYTICS,https://www.acenergy.com.ph/sustainability/,NaN,NaN
3,Acea S.p.A,Italy,NaN,NaN,ISS ESG,https://www.gruppo.acea.it/investitori,NaN,NaN
4,ACS SCE,Spain,April 2018,NaN,VIGEO EIRIS,http://acsindustria.com/,NaN,NaN
...,...,...,...,...,...,...,...,...
610,Yango Group,China,NaN,Appendix to Sustainalytics report,SUSTAINALYTICS,http://www.yango.com.cn/index.php/Ch/Cms/Inves...,NaN,NaN
611,YES BANK Ltd,India,September 2016,KPMG,KPMG,https://www.yesbank.in/annual-reports/fy-2015-...,NaN,NaN
612,Ygrene Energy Fund,US,NaN,Appendix to Sustainalytics report,SUSTAINALYTICS,http://www.sustainalytics.com/sustainability-b...,NaN,NaN
613,Zürcher Kantonalbank (2018),Switzerland,April 2018,oekom research,oekom research,https://www.zkb.ch/de/uu/nb/investor-relations...,NaN,NaN


In [4]:
# generate lists of all companies that have green bonds owned by ECB and all companies that have non-green bonds owned by ECB
companiesECBSet = set(holdings20210402["ISSUER"].tolist()) # set of companies with bonds owned by ECB
greenBondsCompaniesSet = set(greenBonds["Green Bond issuer"].tolist()) # set of companies with green bonds
greenCompanies = [] # this list will hold all companies with green bonds owned by ECB
for company in companiesECBSet:
    for greenBondCompany in greenBondsCompaniesSet:
        # Here, regex are used because names don't exactly match
        if (re.search(".*"+greenBondCompany+".*", company) or re.search(".*"+company+".*",greenBondCompany)):
            greenCompanies.append(company)

# Using OpenFIGI to convert bond ISIN to ticker
FIGIs are unique identifiers of financial instruments issued by Bloomberg. OpenFIGI is an API that maps third-party identifiers to FIGI, but it also returns other information such as company name and ticker.

In [13]:
# Import Packages
import json
import urllib.request
import urllib.parse

In [15]:
# Load Functions
def map_jobs(jobs):
    handler = urllib.request.HTTPHandler()
    opener = urllib.request.build_opener(handler)
    openfigi_url = 'https://api.openfigi.com/v3/mapping'
    request = urllib.request.Request(openfigi_url, data=bytes(json.dumps(jobs), encoding='utf-8'))
    request.add_header('Content-Type','application/json')
    if openfigi_apikey:
        request.add_header('X-OPENFIGI-APIKEY', openfigi_apikey)
    request.get_method = lambda: 'POST'
    connection = opener.open(request)
    if connection.code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return json.loads(connection.read().decode('utf-8'))

def job_results_handler(jobs, job_results):
    df = pd.DataFrame({})
    for job, result in zip(jobs, job_results):
        job_df = pd.DataFrame({'ISIN': [job['idValue']]})
        results_df = pd.read_json(json.dumps(result.get('data', [])))
        df = pd.concat([df, pd.concat([job_df, results_df], axis = 1)])
    return df

In [19]:
# Convert raw data to appropriate input format
ISIN_FIGI = pd.concat([pd.Series(np.tile('ID_ISIN', 1642)), holdings20210402.ISIN], axis = 1)
ISIN_FIGI = ISIN_FIGI.rename(columns = {0: 'idType', 'ISIN': 'idValue'})

# Map to FIGI
openfigi_apikey = 'c89ac66d-e0d2-416f-9c5e-0ed7ec59c770' # This is my personal key (Fred)
jobs_per_access = 100
no_of_access = len(ISIN_FIGI)//jobs_per_access + 1

figi = pd.DataFrame({})
for i in range(no_of_access):
    lower_bound = jobs_per_access * i
    upper_bound = jobs_per_access * (i + 1) if i < no_of_access - 1 else max(ISIN_FIGI.index) + 1
    job = ISIN_FIGI.iloc[lower_bound:upper_bound].to_dict(orient = 'records')
    job_results = map_jobs(job)
    figi = figi.append(job_results_handler(job, job_results))
    
# Extracting the pure ticker
figi.ticker = figi.ticker.apply(lambda x: x.split()[0])
figi.head()

,ISIN,figi,name,ticker,exchCode,compositeFIGI,securityType,marketSector,shareClassFIGI,securityType2,securityDescription
0,BE0002239086,BBG00BH3RNB8,ELIA TRANSMISSION BE,ELIATB,EURONEXT-BRUSS,NaN,EURO MTN,Corp,NaN,Corp,ELIATB 1 3/8 05/27/24
0,BE0002256254,BBG00D8J7316,RESA SA BELGIUM,RESABE,EURONEXT-BRUSS,NaN,EURO-ZONE,Corp,NaN,Corp,RESABE 1 07/22/26
0,BE0002276450,BBG00GCR0947,ELIA TRANSMISSION BE,ELIATB,EURONEXT-BRUSS,NaN,EURO MTN,Corp,NaN,Corp,ELIATB 1 3/8 04/07/27
0,BE0002280494,BBG00GNFY8B8,GRP BRUXELLES LAMBERT SA,GBLBBB,EURONEXT-BRUSS,NaN,EURO-ZONE,Corp,NaN,Corp,GBLBBB 1 3/8 05/23/24
0,BE0002285543,BBG00GW3JTY1,FLUVIUS SYSTEM OP,FLUVIU,EURONEXT-BRUSS,NaN,EURO-ZONE,Corp,NaN,Corp,FLUVIU 2 06/23/25


# Percentage of companies supported by ECB with green ESG scores

Next 4 blocks calculate the score of a given ticker with the function 'web_scraper(ticker)' as shown by the example for Microsoft Corporation (MSFT).
TODO: Find the ticker belonging to each company and iterate over them to see if the ESG score or the Environment Score indicate that the given company is green.

In [14]:
# Source used: https://curt-beck1254.medium.com/scrapping-financial-esg-data-with-python-99d171a12c51
from bs4 import BeautifulSoup
# import pandas as pd
import requests

In [33]:
def web_scraper(ticker):
    elements = []
    web_data = requests.get('https://finance.yahoo.com/quote/'+ticker+'/sustainability?p='+ticker).text
    soup = BeautifulSoup(web_data, 'html.parser')
    esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
    datapoint = esg_score.text if esg_score != None else np.NaN
    controversy_score = soup.find('div', {'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    controversy_datapoint = controversy_score.text if controversy_score != None else np.NaN
    scores = soup.find_all('div', {'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    if len(scores) == 0:
        elements = [np.NaN, np.NaN, np.NaN]
    else:
        for score in scores:
            elements.append(score.text)
        
    df = pd.DataFrame({'Total ESG Score': datapoint,
                      'Environment Score': elements[0],
                       'Social Score': elements[1],
                      'Governance Score': elements[2],
                      'Controversy Score': controversy_datapoint},
                     index=[ticker])
    df = df.astype('float')
    df['Controversy Assessment'] = df.apply(lambda x: level(x['Controversy Score']), axis=1)
    return df

In [16]:
def level(x):
    if x == 0.0:
        return 'No Controversy'
    if x == 1.0:
        return 'Little Controverssy'
    if x == 2.0:
        return 'Moderate Controversy'
    if x == 3.0:
        return 'Relatively High Controversy'
    else:
        return 'Severe Controversy'

In [31]:
web_scraper('MSFT')

ESG score: 15


,Total ESG Score,Environment Score,Social Score,Governance Score,Controversy Score,Controversy Assessment
MSFT,15.0,0.5,9.4,4.9,0.5,Severe Controversy


In [39]:
ESG_Summary = pd.DataFrame({})
for i in set(figi.ticker.iloc[0:50].tolist()):
    ESG_Summary.append(web_scraper(i))
ESG_Summary

""
